Logging performance metrics

In [1]:
from import_data import *
from noise_models import *
from neural_network_basic import *
from Branch_flow_xy_split import *
# from generate_xy import *
import numpy as np
import matplotlib.pyplot as plt
# from volt_to_p_est import *
# from VI_to_P import *
# from linear_models import *
from feature_bins import std_bins_bflow
from IPython.display import clear_output

#%% importing single hour imag dataset
VR, VI, bflow, Pinj, Qinj, CFR, CFI, CTR, CTI = import_curr_branch_data()

In [2]:
def data(addbus=[], output = "vi_all"):
#%% Getting X and Y 
    x_vr, x_vi, x_cr, x_ci,  ytot = Branch_flow_custom_xy_split(bflow, VR, VI, CFR, CFI, CTR, CTI, Pinj, Qinj, output = output,  num_bus = 11, add_bus = addbus)
    #%% Adding noise 
    x_vr_n, x_vi_n = GMM_real_imag(x_vr, x_vi)
    x_cr_n, x_ci_n = GMM_real_imag(x_cr, x_ci)
    # bflow_n = noise_gaussian(X, range1, mu1)
    # bflow_n = NoisyMag(x_bflow.values)
 

    # X_p = VI_to_P(x_vm_n, x_va_n, x_cm_n, x_ca_n, num_bus = 11, add_bus = [])
    X_v = np.concatenate((x_vr_n, x_vi_n, x_cr_n, x_ci_n), axis=1)
    Y_p = ytot
    return X_v, Y_p

In [33]:
X_v, Y_p = data([], "powers")
dnn = Deep_Network(layers=3, nodes=int(X_v.shape[1]*1.25), lr = 1e-3, epoch = 100,
                   batch_norm = True, dropout = 0)

# dnn.y_normalise = True
mse, mape, r2 = dnn.model_parse(X_v,Y_p,Pinj, Qinj, ntest=1)
clear_output()
np.mean(np.sqrt(mse))

4.744218677733563

In [34]:
len(mse[0])

490

In [17]:
np.median(r2[0])

0.41784353541119745

In [39]:
# lengths = [118,118, 186, 186, 186, 186]
lengths = [186*2, 118]
filtered_r2 = [x if -1 <= x <= 1 else 0 for x in r2[0]]
for i in range(len(lengths)):
    start = sum(lengths[:i])
    end = sum(lengths[:i+1])
    print(sum(lengths[:i]), sum(lengths[:i+1]))
    print("mse:", np.sqrt(np.mean(mse[0][  start:end])))
    print("r2:", np.mean(filtered_r2[  start:end]))

0 372
mse: 4.870096957811761
r2: 0.3509191559974188
372 490
mse: 8.46496754206307
r2: 0.11384124675732854


Legacy code, convert mag and ang to real and imaginary

In [4]:
# import numpy as np


# def polar_to_complex(magnitude_array, angle_array_degrees):
#     # Convert angle from degrees to radians
#     angle_array_radians = np.radians(angle_array_degrees)

#     # Calculate real and imaginary components
#     real_part = magnitude_array * np.cos(angle_array_radians)
#     imaginary_part = magnitude_array * np.sin(angle_array_radians)

#     return real_part, imaginary_part

# VR, VI = polar_to_complex(VM, VA)
 
# CFR, CFI = polar_to_complex(CFM, CFA)
# CTR, CTI = polar_to_complex(CTM, CTA)


# np.savetxt("dataset_pf_corrected\V_real.csv", VR, delimiter=',')
# np.savetxt("dataset_pf_corrected\V_imag.csv", VI, delimiter=',')
# np.savetxt("dataset_pf_corrected\CF_real.csv", CFR, delimiter=',')
# np.savetxt("dataset_pf_corrected\CF_imag.csv", CFI, delimiter=',')
# np.savetxt("dataset_pf_corrected\CT_real.csv", CTR, delimiter=',')
# np.savetxt("dataset_pf_corrected\CT_imag.csv", CTI, delimiter=',')